# GEC Data Science Program
## Level 1, Labs 4 & 5

Contents

#### [Lab  4](#Lab-4)
- [Classification](#Classification)
- [Evaluation](#Evaluation)
- [ROC Curve](#ROC-curve)
- [Clustering](#Clustering)

#### [Lab  5](#Lab-5)
- [Feature-Selection](#Feature-Selection)
- [Model-Selection](#Model-Selection)
- [Parameter-Tuning](#Parameter-Tuning)
- [Pipeline](#Pipeline)

### Imports

In [1]:
# from __future__ import division
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn
from sklearn import model_selection, linear_model

In [2]:
from sklearn import svm, model_selection, metrics

In [3]:
from sklearn.tree import DecisionTreeClassifier

In [4]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
%matplotlib inline

### Data Loading

In [6]:
data_path="./"
d = pd.read_csv(data_path+"train_clean.csv", index_col=0)

In [7]:
d.head()

,SibSp,Parch,Fare,Sex_male,Pclass_2,Pclass_3,Embarked_Q,Embarked_S,Age,Survived
PassengerId,,,,,,,,,,
1,0.432550,-0.473408,-0.879247,1.0,0.0,1.0,0.0,1.0,22.0,0
2,0.432550,-0.473408,1.360456,0.0,0.0,0.0,0.0,0.0,38.0,1
3,-0.474279,-0.473408,-0.798092,0.0,0.0,1.0,0.0,1.0,26.0,1
4,0.432550,-0.473408,1.061442,0.0,0.0,0.0,0.0,1.0,35.0,1
5,-0.474279,-0.473408,-0.783739,1.0,0.0,1.0,0.0,1.0,35.0,0


In [8]:
features = list(d.columns)
target = 'Survived'

In [9]:
if target in features:
    features.pop(features.index(target))

In [10]:
features

['SibSp',
 'Parch',
 'Fare',
 'Sex_male',
 'Pclass_2',
 'Pclass_3',
 'Embarked_Q',
 'Embarked_S',
 'Age']

In [11]:
d[features].head()

,SibSp,Parch,Fare,Sex_male,Pclass_2,Pclass_3,Embarked_Q,Embarked_S,Age
PassengerId,,,,,,,,,
1,0.432550,-0.473408,-0.879247,1.0,0.0,1.0,0.0,1.0,22.0
2,0.432550,-0.473408,1.360456,0.0,0.0,0.0,0.0,0.0,38.0
3,-0.474279,-0.473408,-0.798092,0.0,0.0,1.0,0.0,1.0,26.0
4,0.432550,-0.473408,1.061442,0.0,0.0,0.0,0.0,1.0,35.0
5,-0.474279,-0.473408,-0.783739,1.0,0.0,1.0,0.0,1.0,35.0


In [12]:
X=d[features]
y=d[target]

### Data Splitting

In [13]:
train_idx, test_idx = model_selection.train_test_split(d.index, test_size=0.2)

In [14]:
X_train = X.loc[train_idx]
X_test  = X.loc[test_idx]
y_train = y.loc[train_idx]
y_test  = y.loc[test_idx]

# Lab 4

In this lab our goal is to create models to predict which passengers survived Titanic disaster.

## Classification

### Logistic Regression

In [ ]:
lr = linear_model.LogisticRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
y_fit = lr.predict(X_train)
y_pred = lr.predict(X_test)

### Decision Tree

In [5]:
dt = DecisionTreeClassifier()

In [15]:
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [17]:
y_fit_dt = dt.predict(X_train)
y_pred_dt = dt.predict(X_test)

In [20]:
!pip install pydotplus

You are using pip version 9.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Also install GraphViz, and add it to the PATH

https://www.graphviz.org/download/

In [48]:
# !setx PATH "%PATH%;C:\Program Files (x86)\Graphviz2.38\bin"
# !echo %PATH%

In [26]:
import pydotplus
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image 

In [44]:
dot_data = StringIO()
export_graphviz(dt, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)

In [45]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  

In [ ]:
Image(graph.create_png())

## Evaluation

### Q: How do we evaluate a classifier?

In [ ]:
from sklearn import metrics

In [ ]:
print (metrics.accuracy_score(y_test, y_pred))

In [ ]:
true = y_test == y_pred
false = y_test != y_pred

In [ ]:
sum(true), sum(false)

In [ ]:
TP = sum(true &  (y_pred == 1))
TN = sum(true &  (y_pred == 0))

In [ ]:
FP = sum(false & (y_pred == 1))
FN = sum(false & (y_pred == 0))

In [ ]:
print (TN, FP)
print (FN, TP)

In [ ]:
print ("precision (positive predictive value):", TP/(TP+FP))

In [ ]:
print ("recall (sensitivity, true positive rate):", TP/(TP+FN))

In [ ]:
print ("specificity (true negative rate):",TN/(TN+FP))

In [ ]:
print ("False Negative Rate:",FN/(TP+FN))

In [ ]:
print ("False Positive Rate:",FP/(FP+TN))

Usefull information: https://en.wikipedia.org/wiki/Sensitivity_and_specificity

In [ ]:
print (metrics.confusion_matrix(y_test, y_pred, labels=[0, 1]))

In [ ]:
print (metrics.classification_report(y_test, y_pred, target_names=['Died','Survived']))

### Q: The above predictions are based on the default threshold of 0.5. How can we change the threshold?

Hint: Generate prediction probabilities (y_pred_prob) using predict_proba() function.

In [ ]:
y_pred_prob = #add your code here

### Q: Create the following 3 graphs. What can we learn from each?

In [ ]:
plt.hist(y_pred_prob);

In [ ]:
ax = sns.jointplot(y_pred_prob,y_test,kind="kde");
ax.set_axis_labels('predicted probability of survival','survived');

In [ ]:
idx0 = y_test == 0
idx1 = y_test == 1
plt.hist([y_pred_prob[idx0], y_pred_prob[idx1]], bins=15);
plt.legend(['died','survived']);
plt.xlabel("Predicted Probability of Survival");
plt.ylabel("Counts");

### Q: How does the performance metrics change if we change threshold?

Try threshold values of 0.2, 0.5, 0.8 and print classification report for each.

In [ ]:
thr = 0.2
y_pred = y_pred_prob > thr

In [ ]:
print (metrics.classification_report(y_test, y_pred, target_names=['Died','Survived']))

### ROC curve

https://en.wikipedia.org/wiki/Receiver_operating_characteristic 

In [ ]:
fpr, tpr, thr = metrics.roc_curve(y_test, y_pred_prob)

In [ ]:
plt.plot(fpr, tpr,'.-');
plt.xlabel("False Positive Rate");
plt.ylabel("True Positive Rate");
plt.text(fpr[3],tpr[3],"* %0.3f"%thr[3])
plt.text(fpr[-3],tpr[-3],"* %0.3f"%thr[-3])

In [ ]:
auc = metrics.auc(fpr, tpr)
print ("AUC:",auc)

### Scatter plot visualization of true and false predictions

In [ ]:
from sklearn import decomposition

pca = decomposition.PCA()
X_pca = pca.fit_transform(X)
tst_pca = X_pca[test_idx-1]

In [ ]:
thr = 0.5
y_pred = y_pred_prob > thr
plt.figure(figsize=(10,7))
plt.scatter(tst_pca[y_test==0,0],tst_pca[y_test==0,1], marker='x', c=y_pred[y_test==0], 
            cmap=plt.cm.rainbow, alpha=0.7);
plt.scatter(tst_pca[y_test==1,0],tst_pca[y_test==1,1], marker='o', c=y_pred[y_test==1], 
            cmap=plt.cm.rainbow, alpha=0.7);
plt.colorbar();
print ("o: survived")
print ("x: died")
print ("red: predicted survived")
print ("purple: predicted dead")
print ("TP: red O")
print ("TN: purple X")

### SVM

In [ ]:
from sklearn import svm

In [ ]:
svm_model = svm.SVC()

In [ ]:
svm_model.fit(X_train, y_train)

In [ ]:
y_pred = svm_model.predict(X_test)

In [ ]:
print (metrics.classification_report(y_test, y_pred, target_names=['Died','Survived']))

In [ ]:
y_pred_score = svm_model.decision_function(X_test)

In [ ]:
plt.hist(y_pred_score);

In [ ]:
y_pred_prob2 = (y_pred_score - min(y_pred_score))/(max(y_pred_score)-min(y_pred_score))

In [ ]:
plt.hist(y_pred_prob2);

In [ ]:
idx0 = y_test == 0
idx1 = y_test == 1
plt.hist([y_pred_prob2[idx0], y_pred_prob2[idx1]], bins=15);
plt.legend(['died','survived']);
plt.xlabel("Predicted Probability of Survival");
plt.ylabel("Counts");

In [ ]:
fpr2, tpr2, thr2 = metrics.roc_curve(y_test, y_pred_prob2)

In [ ]:
auc2 = metrics.auc(fpr2,tpr2)

In [ ]:
plt.plot(fpr, tpr);
plt.plot(fpr2, tpr2);
plt.xlabel("False Positive Rate");
plt.ylabel("True Positive Rate");

plt.legend(["Linear Regression (auc=%0.2f)"%auc,"SVM (auc=%0.2f)"%auc2]);

In [ ]:
print ("AUC:",auc2)

### Random Forest

### *Q: Create ROC curve for a Random Forest model and compare with the other two.

In [ ]:
from skle import 

## Clustering

http://scikit-learn.org/stable/modules/clustering.html

In [ ]:
from sklearn import cluster

### Kmeans

In [ ]:
km = cluster.KMeans(n_clusters=2, tol=1e-99, max_iter=999, verbose=0)

In [ ]:
km.fit(X_train,y_train)

In [ ]:
y_km = km.fit_predict(X_test)

In [ ]:
plt.hist(y_km,bins=8);

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(tst_pca[y_test==0,0],tst_pca[y_test==0,1], marker='x', c=y_km[y_test==0], cmap=plt.cm.rainbow, alpha=0.7);
plt.scatter(tst_pca[y_test==1,0],tst_pca[y_test==1,1], marker='o', c=y_km[y_test==1], cmap=plt.cm.rainbow, alpha=0.7);
plt.colorbar();

### DBSCAN

In [ ]:
dbs = cluster.DBSCAN(1)

In [ ]:
y_dbs = dbs.fit_predict(X_test)

In [ ]:
plt.hist(y_dbs);

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(tst_pca[y_test==0,0],tst_pca[y_test==0,1], marker='x', c=y_dbs[y_test==0], cmap=plt.cm.rainbow, alpha=0.7);
plt.scatter(tst_pca[y_test==1,0],tst_pca[y_test==1,1], marker='o', c=y_dbs[y_test==1], cmap=plt.cm.rainbow, alpha=0.7);
plt.colorbar();

# Lab 5

In this lab we talk about some advanced topics to improve our predictions.

## Feature Selection

Which feature are more important and which ones are not important?

http://scikit-learn.org/stable/modules/feature_selection.html

In [ ]:
from sklearn import feature_selection

### SelectFromModel

In [ ]:
mdl = svm.LinearSVC(penalty='l1', dual=False).fit(X_train, y_train)

In [ ]:
#np.sort(np.abs(mdl.coef_))

In [ ]:
feature_selector = feature_selection.SelectFromModel(estimator=mdl, prefit=True, threshold=0.1)

In [ ]:
X_train1 = feature_selector.transform(X_train)

In [ ]:
pd.DataFrame(X_train1).head()

In [ ]:
X_train.head()

In [ ]:
print ("Original number of features:",X_train.shape[1])
print ("Selected number of features:",X_train1.shape[1])

### Recursive Feature Elimination (RFE)

In [ ]:
rfe = feature_selection.RFE(estimator=mdl, verbose=1)

In [ ]:
rfe.fit(X_train,y_train)

In [ ]:
sorted(zip(rfe.ranking_, X_train.columns))

## Over-fitting, Under-fitting

In [ ]:
train_score = []
test_score = []
c_range = np.logspace(-3,4,50) 
for c in c_range:
    #print ('.',)
    mdl = svm.SVC(C=c)
    mdl.fit(X_train, y_train)
    y_fit = mdl.predict(X_train)
    y_pred = mdl.predict(X_test)
    train_score.append(metrics.f1_score(y_train, y_fit))
    test_score.append(metrics.f1_score(y_test, y_pred))

In [ ]:
plt.semilogx(c_range[:len(train_score)], train_score,'.-');
plt.semilogx(c_range[:len(train_score)], test_score,'.-');
plt.legend(['train','test'])
plt.xlabel('parameter C')
plt.ylabel('F1-score')

### Q: Given the above graph, what's the best value of C?

In [ ]:
#Chan Yu
c_range[:len(train_score)][np.argmax(test_score)]

In [ ]:
best_c = c_range[np.argmax(test_score)]
print ("Best C=",best_c)
print ("Best F1-score:",np.max(test_score))

### Q: Which data is better to use for parameter tuning: test data or validation data?

## Model Selection

### Cross-Validation
http://scikit-learn.org/stable/modules/cross_validation.html

In [ ]:
from sklearn import model_selection

In [ ]:
mdl = svm.SVC(C=1.0)
scores = model_selection.cross_val_score(mdl, X, y, cv=5, verbose=1, n_jobs=5, scoring="f1")

In [ ]:
scores

In [ ]:
print("Cross-Validated F1-Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() ))

### Cross-validation Iterators
#### K-fold

In [ ]:
kf = model_selection.KFold(n_splits=5)
mdl = svm.SVC(C=1)

train_score = []
test_score = []
for train_idx, test_idx in kf.split(X):
    #print '.',
    X_trn = X.values[train_idx]
    X_tst = X.values[test_idx]
    y_trn = y.values[train_idx]
    y_tst = y.values[test_idx]
    mdl.fit(X_trn, y_trn)
    y_fit = mdl.predict(X_trn)
    y_pred = mdl.predict(X_tst)
    train_score.append(metrics.f1_score(y_trn, y_fit))
    test_score.append(metrics.f1_score(y_tst, y_pred))
   

In [ ]:
print("Cross-Validated Train F1-Score: %0.2f (+/- %0.2f)" % (np.mean(train_score), np.std(train_score) ))

In [ ]:
print("Cross-Validated Test F1-Score: %0.2f (+/- %0.2f)" % (np.mean(test_score), np.std(test_score) ))

In [ ]:
plt.hist([train_score, test_score]);
plt.legend(['train','test']);

#### StratifiedKFold

StratifiedKFold can be used to balance the splits if labels are unbalanced.

In [ ]:
kf = model_selection.StratifiedKFold(n_splits=50)
mdl = svm.SVC(C=1)

train_score = []
test_score = []
for train_idx, test_idx in kf.split(X, y):
    #print '.',
    X_trn = X.values[train_idx]
    X_tst = X.values[test_idx]
    y_trn = y.values[train_idx]
    y_tst = y.values[test_idx]
    mdl.fit(X_trn, y_trn)
    y_fit = mdl.predict(X_trn)
    y_pred = mdl.predict(X_tst)
    train_score.append(metrics.f1_score(y_trn, y_fit))
    test_score.append(metrics.f1_score(y_tst, y_pred))

In [ ]:
print("Cross-Validated Train F1-Score: %0.2f (+/- %0.2f)" % (np.mean(train_score), np.std(train_score) ))

In [ ]:
print("Cross-Validated Test F1-Score: %0.2f (+/- %0.2f)" % (np.mean(test_score), np.std(test_score) ))

In [ ]:
plt.hist([train_score, test_score]);
plt.legend(['train','test']);

### Parameter Tuning
http://scikit-learn.org/stable/modules/grid_search.html

In [ ]:
def report(clf):
    print("Best parameters set found on development set:")
    print()
    print (clf.best_params_)
    print()
    print ("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print ("Detailed classification report:")
    print()
    print ("The model is trained on the full development set.")
    print ("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print (metrics.classification_report(y_true, y_pred))
    print()

#### Exhaustive Grid Search

Tip: split data to development_set (to be fed to the GridSearchCV instance) and an evaluation_set

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.5, random_state=0)

In [ ]:
#not recommended. takes a long time.
# tuned_parameters = {'kernel': ['rbf','linear'], 'C': [0.1, 1, 10]}
# clf = model_selection.GridSearchCV(svm.SVC(), tuned_parameters, verbose = 2, n_jobs=-1)
# clf.fit(X_train, y_train)
# report(clf)

#### RandomizedSearchCV

In [ ]:
tuned_parameters = {'kernel': ['rbf','linear'], 
                    'C': [0.01, 0.1, 1, 10, 100], 
                    'gamma': ['auto',0.01, 1]}

random_search = model_selection.RandomizedSearchCV(svm.SVC(), verbose= 2,
                                                   param_distributions=tuned_parameters, 
                                                   n_iter=3)

random_search.fit(X, y)

In [ ]:
report(random_search)

## Pipelines

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

In [ ]:
clf = linear_model.LogisticRegression()
sc = preprocessing.StandardScaler()
pipe_model = Pipeline([('scaler', sc), ('lr', clf)])
pipe_model.fit(X_train, y_train)

In [ ]:
y_pred = pipe_model.predict(X_test)

In [ ]:
print (metrics.classification_report(y_test,y_pred))